# Практическое задание к уроку 12. Модель Transformer-2

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install torchmetrics
!pip install rouge
!pip install torch
import torch
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 25.0 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 6.6 MB 38.1 MB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 9.8 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
     |████████████████████████████████| 140 kB 53.5 MB/s 
     |████████████████████████████████| 212 kB 55.2 MB/s 
     |████████████████████████████

In [2]:
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")

No config specified, defaulting to: gazeta/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 52400
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5770
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5265
    })
})

In [4]:
import pandas as pd
df_train = pd.DataFrame(dataset['train'])
df_train.head()

,text,summary,title,date,url
0,«По итогам 2011 года чистый отток может состав...,"В 2011 году из России уйдет $80 млрд, считают ...",Прогноз не успевает за оттоком,2011-11-30 18:33:39,https://www.gazeta.ru/financial/2011/11/30/385...
1,Российское подразделение интернет-корпорации G...,"Юлия Соловьева, экс-директор холдинга «Профмед...",Google закончил поиск,2013-01-24 18:20:09,https://www.gazeta.ru/business/2013/01/24/4939...
2,Басманный районный суд Москвы вечером 6 феврал...,Суд арестовал на два месяца четверых экс-чинов...,«Фигуранты дела могут давить на свидетелей»,2018-02-06 21:21:14,https://www.gazeta.ru/social/2018/02/06/116393...
3,Как повлияло вступление в ВТО на конкурентносп...,Мнения предпринимателей по поводу вступления в...,«С последних традиционно «отжимают» больше»,2013-06-21 17:43:50,https://www.gazeta.ru/business/2013/06/21/5388...
4,К третьему сезону «Голос» на Первом канале ста...,На Первом канале завершился третий сезон шоу «...,Третий «Голос» за Градского,2014-12-27 01:10:01,https://www.gazeta.ru/culture/2014/12/27/a_636...


In [5]:
df_val = pd.DataFrame(dataset['validation'])
df_val.head()

,text,summary,title,date,url
0,"После громких приобретений Андре Шюррле, Гуса ...",Московский «Спартак» продолжает активную транс...,Замена Фернандо: кого может купить «Спартак»,2019-08-11 23:14:58,https://www.gazeta.ru/sport/2019/08/11/a_12572...
1,Американское издание The National Interest оце...,Издание The National Interest оценило перспект...,Шестое поколение: в США оценили российский бом...,2019-08-11 21:16:04,https://www.gazeta.ru/army/2019/08/11/12572317...
2,Министр иностранных дел России Сергей Лавров с...,"Глава МИД России Сергей Лавров заявил, что в 2...",«Зачем гонять людей?»: США требуют новый рефер...,2019-06-06 09:47:35,https://www.gazeta.ru/politics/2019/06/06_a_12...
3,Минфин предложил с января 2020 года увеличить ...,Министерство финансов предлагает вдвое поднять...,Перейти на отечественное: что будет с зарубежн...,2019-07-24 20:38:15,https://www.gazeta.ru/business/2019/07/24/1252...
4,Заявление командующего военно-воздушными силам...,Американские ПВО провалились в Саудовской Арав...,«Глупое заявление»: Медведев напомнил о провал...,2019-09-20 21:16:22,https://www.gazeta.ru/army/2019/09/20/12664933...


In [6]:
df_test = pd.DataFrame(dataset['test'])
df_test.head()

,text,summary,title,date,url
0,Американское аэрокосмическое агентство NASA ог...,"В NASA назвали четыре миссии в дальний космос,...","Венера, Ио или Тритон: куда полетит NASA",2020-02-14 16:39:11,https://www.gazeta.ru/science/2020/02/14_a_129...
1,Около 11 тысяч зрителей увидели все самое лучш...,25 и 26 февраля в Кремлевском дворце съездов п...,«Люди в Бурятии очень талантливые»,2020-02-28 10:44:13,https://www.gazeta.ru/social/2020/02/28/129806...
2,7 ноября в Белоруссии прошли выборы членов сов...,В Белоруссии в день годовщины Октябрьской рево...,Вспомнить СССР: как Лукашенко провел выборы,2019-11-07 19:55:08,https://www.gazeta.ru/politics/2019/11/07_a_12...
3,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,2020-03-01 16:50:06,https://www.gazeta.ru/culture/2020/03/01/a_129...
4,Депутат Верховной рады от партии «Слуга народа...,Украина не должна выплачивать пенсии жителям Д...,«Поддерживают Россию»: почему Киев не платит п...,2020-02-06 12:41:24,https://www.gazeta.ru/business/2020/02/06/1294...


#### Предобученная модель summarization

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "IlyaGusev/rugpt3medium_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

article_text = "..."

text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False, 
    padding=False,
    truncation=True
)["input_ids"]
input_ids = text_tokens + [tokenizer.sep_token_id]
input_ids = torch.LongTensor([input_ids])

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)

summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
summary = summary.split(tokenizer.sep_token)[1]
summary = summary.split(tokenizer.eos_token)[0]
print(summary)

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/105 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/986 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Нападающий «Зенита» Александр Кокорин и полузащитник «Краснодара» Павел Мамаев, обвиняемые в избиении водителя ведущей Первого канала Ольги Ушаковой, отпущены под подписку о невыезде. Об этом сообщил адвокат Кокорина Игорь Бушманов.


In [9]:
#функция подсчета метрик
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [10]:
#подсчет метрик на примере предсказания модели
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
target = sent_tokenize(df_test['summary'][1])
preds = sent_tokenize(summary)
calc_scores(target, preds)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Count: 2
Ref: Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: Об этом сообщил адвокат Кокорина Игорь Бушманов.
BLEU:  0.05879762398503692
ROUGE:  {'rouge-1': {'r': 0.07142857142857142, 'p': 0.043478260869565216, 'f': 0.05405405170197235}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.07142857142857142, 'p': 0.043478260869565216, 'f': 0.05405405170197235}}


**Вывод:** Результат слабый.

#### Обучение модели для генерации заголовков

In [11]:
#добавляем токены начала и конца предложения
df_train['title_clean'] = df_train['title'].apply(lambda v: 'BOS ' + v + ' EOS')
df_test['title_clean'] = df_test['title'].apply(lambda v: 'BOS ' + v + ' EOS')

In [12]:
df_test['title_clean']

0        BOS Венера, Ио или Тритон: куда полетит NASA EOS
1              BOS «Люди в Бурятии очень талантливые» EOS
2       BOS Вспомнить СССР: как Лукашенко провел выбор...
3       BOS «Он очень переживал»: Бабкина об отношения...
4       BOS «Поддерживают Россию»: почему Киев не плат...
                              ...                        
5765    BOS Новая «Игра престолов»? Netflix показал тр...
5766    BOS «Это было ужасно»: Джоли эвакуировали со с...
5767    BOS «Это камни»: ученого затравили за змей на ...
5768    BOS «До 200 тысяч зараженных»: как распростран...
5769    BOS С российским реактором: Китай создаст атом...
Name: title_clean, Length: 5770, dtype: object

In [13]:
#уменьшаем объем датасетов
MAX_TRAIN_SAMPLE = 1000
MAX_TEST_SAMPLE = 200

df_train = df_train[:MAX_TRAIN_SAMPLE]
df_test = df_test[:MAX_TEST_SAMPLE]

In [14]:
max_len_text = 700
max_len_sum = 50

In [15]:
tok_text = Tokenizer()
#создаем словарь индексов на основе частоты слов в df_train['text'] (чем меньше число, тем чаще встречается слово)
tok_text.fit_on_texts(df_train['text'])
#преобразуем каждый text из df_train['text'] в последовательность целых чисел  в соответсвии со словарем
x_train_tok = tok_text.texts_to_sequences(df_train['text'])
x_test_tok = tok_text.texts_to_sequences(df_test['text'])

In [16]:
text_vocab_size=len(tok_text.word_index)+1
#приводим последовательности целых чисел к одной длине:
#к каждому элементу из x_train_tok добавляем (отнимаем) до max_len_text (от max_len_text) нулями сзади
padded_x_train = pad_sequences(x_train_tok, maxlen=max_len_text, padding='post', truncating='post')
padded_x_test = pad_sequences(x_test_tok, maxlen=max_len_text, padding='post', truncating='post')

In [17]:
padded_x_train.shape

(1000, 700)

In [18]:
tok_sum = Tokenizer()
#создаем словарь индексов на основе частоты слов в df_train['title_clean']
tok_sum.fit_on_texts(df_train['title_clean'])
#преобразуем каждый text из df_train['title_clean'] в последовательность целых чисел  в соответсвии со словарем
x_train_sum = tok_sum.texts_to_sequences(df_train['title_clean'])
x_test_sum = tok_sum.texts_to_sequences(df_test['title_clean'])

In [19]:
sum_vocab_size=len(tok_sum.word_index)+1
#приводим последовательности целых чисел к одной длине:
#к каждому элементу из x_train_tok добавляем (отнимаем) до max_len_sum (от max_len_sum) нулями сзади
padded_x_train_sum = pad_sequences(x_train_sum, maxlen=max_len_sum, padding='post', truncating='post')
padded_x_test_sum = pad_sequences(x_test_sum, maxlen=max_len_sum, padding='post', truncating='post')

In [20]:
padded_x_train_sum.shape

(1000, 50)

In [21]:
#словарь индекс: токен для tok_text
reverse_text_index=tok_text.index_word
#словарь индекс: токен для tok_sum
reverse_sum_index=tok_sum.index_word
#словарь токен: индекс для tok_sum
sum_wordindex=tok_sum.word_index

In [22]:
from tensorflow.keras.layers import GRU, AdditiveAttention

In [23]:
BUFFER_SIZE = len(padded_x_train)
BATCH_SIZE = 32
steps_per_epoch = len(padded_x_train)//BATCH_SIZE

#каждую пару (padded_x_train, padded_x_train_sum) преобразуем в тензор
dataset = tf.data.Dataset.from_tensor_slices((padded_x_train, padded_x_train_sum)).shuffle(BUFFER_SIZE)
#объединяем последоваетльные элементы в батчи, неполный батч удаляем
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [24]:
for element in dataset:
  print(element)
  break

(<tf.Tensor: shape=(32, 700), dtype=int32, numpy=
array([[    1,   384,  1996, ...,     0,     0,     0],
       [ 4085,     1, 11896, ...,    77,     5,   195],
       [   40,  7532,  2149, ...,     0,     0,     0],
       ...,
       [ 2909,    29,    72, ...,     0,     0,     0],
       [ 3406,     8,  2147, ...,     0,     0,     0],
       [  678,    26,  2017, ..., 36140,  6619, 11224]], dtype=int32)>, <tf.Tensor: shape=(32, 50), dtype=int32, numpy=
array([[   1,  523,    3, ...,    0,    0,    0],
       [   1, 1713,  201, ...,    0,    0,    0],
       [   1,   66,    6, ...,    0,    0,    0],
       ...,
       [   1, 1017,    3, ...,    0,    0,    0],
       [   1,  147,    5, ...,    0,    0,    0],
       [   1,  132, 1970, ...,    0,    0,    0]], dtype=int32)>)


In [25]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 700]), TensorShape([32, 50]))

In [26]:
# Построение модели (Encoder и Decoder)

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru1(x, initial_state = hidden)
        output, state = self.gru2(output, initial_state = state)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, x, query, value):
        # enc_output shape == (batch_size, max_length, hidden_size)
        #attention_weights = self.attention([ tf.expand_dims(query, 1), value,])
        context_vector = self.attention([tf.expand_dims(query, 1), value,])
        #context_vector = tf.squeeze(context_vector)


        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([context_vector, x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [27]:
latent_dim = 300
embedding_dim=200

#создаем экземпляры encoder и decoder
encoder = Encoder(text_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)
decoder = Decoder(sum_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

In [28]:
sample_output

<tf.Tensor: shape=(32, 700, 300), dtype=float32, numpy=
array([[[-1.63050797e-02,  4.27322258e-04,  1.70327257e-03, ...,
          5.48622850e-03, -1.62952812e-03, -4.01851209e-03],
        [-4.21321113e-03,  6.48785243e-03,  5.44484239e-03, ...,
          3.67857143e-03, -1.60099880e-03, -2.37860018e-03],
        [-4.07086220e-03,  4.78431489e-03,  5.33468509e-03, ...,
          7.11200200e-03, -7.77012086e-04, -1.72228401e-03],
        ...,
        [-1.96394557e-03,  2.80814013e-03, -5.86491404e-03, ...,
         -3.28336796e-03, -2.23142980e-03,  3.28090461e-03],
        [-3.32718692e-03, -1.74666662e-03, -7.50378519e-03, ...,
         -2.59716040e-03, -8.00449960e-03,  2.13957299e-03],
        [-2.18682596e-03, -1.24034588e-03, -4.72728483e-04, ...,
         -3.28176538e-03, -8.19528475e-03,  2.75540189e-03]],

       [[-2.17261445e-03,  1.91188988e-03,  7.99154583e-03, ...,
         -4.04560054e-03, -2.11069118e-02, -4.81959386e-03],
        [-1.93328306e-03,  1.29631034e-03,  4.6

In [29]:
sample_hidden

<tf.Tensor: shape=(32, 300), dtype=float32, numpy=
array([[-2.18682596e-03, -1.24034588e-03, -4.72728483e-04, ...,
        -3.28176538e-03, -8.19528475e-03,  2.75540189e-03],
       [-1.58108585e-02, -9.38311033e-03, -7.95544498e-03, ...,
        -1.05700186e-02, -2.09018309e-02,  1.18085593e-02],
       [-1.58108603e-02, -9.38311219e-03, -7.95544405e-03, ...,
        -1.05700195e-02, -2.09018346e-02,  1.18085593e-02],
       ...,
       [-8.60042055e-05, -3.58116603e-03, -2.79945415e-03, ...,
        -9.67608951e-03, -1.15786144e-03, -9.45393089e-03],
       [-1.58108585e-02, -9.38311219e-03, -7.95544311e-03, ...,
        -1.05700167e-02, -2.09018327e-02,  1.18085612e-02],
       [-1.58108585e-02, -9.38311219e-03, -7.95544405e-03, ...,
        -1.05700186e-02, -2.09018327e-02,  1.18085602e-02]], dtype=float32)>

In [30]:
# Компиляция модели

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    # определяем ненулевые элементы
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    #переводим mask к типу loss_
    mask = tf.cast(mask, dtype=loss_.dtype)
    #умножаем loss на маску
    loss_ *= mask

    return tf.reduce_mean(loss_) #возвращаем среднее значение элементов тензора

In [31]:
# Обучение модели

@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([sum_wordindex['bos']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [32]:
import unicodedata
import re
import numpy as np
import os
import io
import time

checkpoint_dir = './training_summ_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [33]:
EPOCHS = 300

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 10 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.9549
Epoch 1 Batch 10 Loss 0.8573
Epoch 1 Batch 20 Loss 0.8542
Epoch 1 Batch 30 Loss 0.7608
Epoch 1 Loss 0.8961
Time taken for 1 epoch 67.11566138267517 sec

Epoch 2 Batch 0 Loss 0.8565
Epoch 2 Batch 10 Loss 0.7780
Epoch 2 Batch 20 Loss 0.7961
Epoch 2 Batch 30 Loss 0.7593
Epoch 2 Loss 0.8261
Time taken for 1 epoch 9.750741243362427 sec

Epoch 3 Batch 0 Loss 0.7377
Epoch 3 Batch 10 Loss 0.8266
Epoch 3 Batch 20 Loss 0.8229
Epoch 3 Batch 30 Loss 0.8595
Epoch 3 Loss 0.7942
Time taken for 1 epoch 8.85076642036438 sec

Epoch 4 Batch 0 Loss 0.8271
Epoch 4 Batch 10 Loss 0.7618
Epoch 4 Batch 20 Loss 0.7835
Epoch 4 Batch 30 Loss 0.7453
Epoch 4 Loss 0.7732
Time taken for 1 epoch 11.14905047416687 sec

Epoch 5 Batch 0 Loss 0.8021
Epoch 5 Batch 10 Loss 0.8306
Epoch 5 Batch 20 Loss 0.7276
Epoch 5 Batch 30 Loss 0.7690
Epoch 5 Loss 0.7718
Time taken for 1 epoch 9.781400203704834 sec

Epoch 6 Batch 0 Loss 0.6588
Epoch 6 Batch 10 Loss 0.8188
Epoch 6 Batch 20 Loss 0.6780
Epoch 6 Ba

In [34]:
# Оценка модели

def evaluate(sentence):
    inputs = [tok_text.word_index[i] for i in sentence.split(' ') if i !='']
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_len_text,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, latent_dim))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tok_sum.word_index['bos']], 0)

    for t in range(max_len_sum):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += tok_sum.index_word[predicted_id] + ' '

        if tok_sum.index_word[predicted_id] == 'eos':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [35]:
def summ(sentence):
    result, sentence = evaluate(sentence)

    return result

In [36]:
#функция перевода последовательности индексов в текст
def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_text_index[i]+' '
    return newString
#функция перевода последовательности индексов в title
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=sum_wordindex['bos']) and i!=sum_wordindex['eos']):
            newString=newString+reverse_sum_index[i]+' '
    return newString

In [37]:
for i in range(5):
    text2 = seq2text(padded_x_test[i])
    print("Original title:", seq2summary(padded_x_test_sum[i]))
    print("Predicted title: ", summ(text2.strip()))
    print("\n")

Original title: или куда 
Predicted title:  «в команде есть равнодушные люди» eos 


Original title: «люди в 
Predicted title:  дело рук машкова сына табакова выдавили из «табакерки» eos 


Original title: ссср как выборы 
Predicted title:  через хука к войне eos 


Original title: «он с 
Predicted title:  преступные связи мэр дзержинского попался на взятке eos 


Original title: почему не пенсии 
Predicted title:  dst покупает китайский amazon eos 




**Вывод:** Странный результат, но контекст прогнозируемоего названия понятен.